<a href="https://colab.research.google.com/github/leejukyu/Deeplearning/blob/main/Lec02_WideDeepRegression.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
 ## Codes are adapted from SungKim@HKUST

In [ ]:
from google.colab import drive
drive.mount('/content/drive', force_remount=True)

Mounted at /content/drive


In [ ]:
cd '/content/drive/MyDrive/'

/content/drive/MyDrive


## 01. Wide and Deep Model (Diabets_logistic)

In [ ]:
import torch
from torch.autograd import Variable
import numpy as np

In [ ]:
xy = np.loadtxt('CNNPytorchCodes/data/diabetes.csv', delimiter=',', dtype=np.float32)
x_data = Variable(torch.from_numpy(xy[:, 0:-1]))
y_data = Variable(torch.from_numpy(xy[:, [-1]]))

print(x_data.data.shape)
print(y_data.data.shape)

torch.Size([759, 8])
torch.Size([759, 1])


In [ ]:
class Model(torch.nn.Module):

    def __init__(self):
        """
        In the constructor we instantiate two nn.Linear module
        """
        super(Model, self).__init__()
        self.l1 = torch.nn.Linear(8, 6)
        self.l2 = torch.nn.Linear(6, 4)
        self.l3 = torch.nn.Linear(4, 1)

        self.sigmoid = torch.nn.Sigmoid()

    def forward(self, x):
        """
        In the forward function we accept a Variable of input data and we must return
        a Variable of output data. We can use Modules defined in the constructor as
        well as arbitrary operators on Variables.
        """
        out1 = self.sigmoid(self.l1(x))
        out2 = self.sigmoid(self.l2(out1))
        y_pred = self.sigmoid(self.l3(out2))
        return y_pred

# our model
model = Model()

In [ ]:
# Construct our loss function and an Optimizer. The call to model.parameters()
# in the SGD constructor will contain the learnable parameters of the two
# nn.Linear modules which are members of the model.

criterion = torch.nn.BCELoss(size_average=True)
optimizer = torch.optim.SGD(model.parameters(), lr=0.1)

/usr/local/lib/python3.7/dist-packages/torch/nn/_reduction.py:42: UserWarning: size_average and reduce args will be deprecated, please use reduction='mean' instead.
  warnings.warn(warning.format(ret))


In [ ]:
# Training loop
for epoch in range(100):
        # Forward pass: Compute predicted y by passing x to the model
    y_pred = model(x_data)

    # Compute and print loss
    loss = criterion(y_pred, y_data)
    print(epoch, loss.item())

    # Zero gradients, perform a backward pass, and update the weights.
    optimizer.zero_grad()
    loss.backward()
    optimizer.step()

0 0.6850830316543579
1 0.6812351942062378
2 0.6777719855308533
3 0.6746554374694824
4 0.6718510389328003
5 0.6693276166915894
6 0.6670571565628052
7 0.6650142073631287
8 0.6631759405136108
9 0.6615219712257385
10 0.6600335240364075
11 0.6586939692497253
12 0.6574884057044983
13 0.6564033031463623
14 0.655426561832428
15 0.6545471549034119
16 0.6537553071975708
17 0.6530423760414124
18 0.6524002552032471
19 0.6518219113349915
20 0.6513009071350098
21 0.6508314609527588
22 0.6504085063934326
23 0.650027334690094
24 0.6496837735176086
25 0.6493740081787109
26 0.6490947008132935
27 0.6488428711891174
28 0.6486157178878784
29 0.6484108567237854
30 0.6482259035110474
31 0.648059070110321
32 0.6479084491729736
33 0.647772490978241
34 0.6476497054100037
35 0.6475387811660767
36 0.6474385261535645
37 0.6473480463027954
38 0.6472660899162292
39 0.6471920013427734
40 0.6471250653266907
41 0.6470645070075989
42 0.6470096707344055
43 0.6469599604606628
44 0.6469149589538574
45 0.6468741297721863
46

## 02. Diabet Classification Using Custom DataLoader

In [ ]:
# References
# https://github.com/yunjey/pytorch-tutorial/blob/master/tutorials/01-basics/pytorch_basics/main.py
# http://pytorch.org/tutorials/beginner/data_loading_tutorial.html#dataset-class
import torch
import numpy as np
from torch.autograd import Variable
from torch.utils.data import Dataset, DataLoader

In [ ]:
class DiabetesDataset(Dataset):
    """ Diabetes dataset."""

    # Initialize your data, download, etc.
    def __init__(self):
        xy = np.loadtxt('./data/diabetes.csv.gz',
                        delimiter=',', dtype=np.float32)
        self.len = xy.shape[0]
        self.x_data = torch.from_numpy(xy[:, 0:-1])
        self.y_data = torch.from_numpy(xy[:, [-1]])

    def __getitem__(self, index):
        return self.x_data[index], self.y_data[index]

    def __len__(self):
        return self.len

In [ ]:
dataset = DiabetesDataset()
train_loader = DataLoader(dataset=dataset,
                          batch_size=32,
                          shuffle=True,
                          num_workers=2)

In [ ]:
class Model(torch.nn.Module):

    def __init__(self):
        """
        In the constructor we instantiate two nn.Linear module
        """
        super(Model, self).__init__()
        self.l1 = torch.nn.Linear(8, 6)
        self.l2 = torch.nn.Linear(6, 4)
        self.l3 = torch.nn.Linear(4, 1)

        self.sigmoid = torch.nn.Sigmoid()

    def forward(self, x):
        """
        In the forward function we accept a Variable of input data and we must return
        a Variable of output data. We can use Modules defined in the constructor as
        well as arbitrary operators on Variables.
        """
        out1 = self.sigmoid(self.l1(x))
        out2 = self.sigmoid(self.l2(out1))
        y_pred = self.sigmoid(self.l3(out2))
        return y_pred

# our model
model = Model()

In [ ]:
# Construct our loss function and an Optimizer. The call to model.parameters()
# in the SGD constructor will contain the learnable parameters of the two
# nn.Linear modules which are members of the model.
criterion = torch.nn.BCELoss(size_average=True)
optimizer = torch.optim.SGD(model.parameters(), lr=0.1)

/usr/local/lib/python3.7/dist-packages/torch/nn/_reduction.py:42: UserWarning: size_average and reduce args will be deprecated, please use reduction='mean' instead.
  warnings.warn(warning.format(ret))


In [ ]:
# Training loop
for epoch in range(2):
    for i, data in enumerate(train_loader, 0):
        # get the inputs
        inputs, labels = data

        # wrap them in Variable
        inputs, labels = Variable(inputs), Variable(labels)

        # Forward pass: Compute predicted y by passing x to the model
        y_pred = model(inputs)

        # Compute and print loss
        loss = criterion(y_pred, labels)
        print(epoch, i, loss.item())

        # Zero gradients, perform a backward pass, and update the weights.
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

0 0 0.6928312182426453
0 1 0.6921122074127197
0 2 0.6880536079406738
0 3 0.6801295280456543
0 4 0.6746836304664612
0 5 0.68610018491745
0 6 0.6735557913780212
0 7 0.6752679347991943
0 8 0.6570809483528137
0 9 0.6714505553245544
0 10 0.6776877641677856
0 11 0.6772294044494629
0 12 0.6846486926078796
0 13 0.6462717652320862
0 14 0.6157056093215942
0 15 0.6139795184135437
0 16 0.6406874656677246
0 17 0.627046525478363
0 18 0.6877992153167725
0 19 0.612067461013794
0 20 0.7020623087882996
0 21 0.6881701350212097
0 22 0.611191987991333
0 23 0.6691051721572876
1 0 0.6891078352928162
1 1 0.6754580736160278
1 2 0.6482507586479187
1 3 0.6756840348243713
1 4 0.647898256778717
1 5 0.6615686416625977
1 6 0.6612523794174194
1 7 0.6472554802894592
1 8 0.704450786113739
1 9 0.5918007493019104
1 10 0.6461097002029419
1 11 0.5851393938064575
1 12 0.6446921229362488
1 13 0.6936346888542175
1 14 0.5804551243782043
1 15 0.694953441619873
1 16 0.5776549577713013
1 17 0.6086737513542175
1 18 0.6795974969863

## 03. Softmax Loss

In [ ]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torchvision import datasets, transforms
from torch.autograd import Variable

In [ ]:
# Cross entropy example
import numpy as np
# One hot
# 0: 1 0 0
# 1: 0 1 0
# 2: 0 0 1
Y = np.array([1, 0, 0])

Y_pred1 = np.array([0.7, 0.2, 0.1])
Y_pred2 = np.array([0.1, 0.3, 0.6])
print("loss1 = ", np.sum(-Y * np.log(Y_pred1)))
print("loss2 = ", np.sum(-Y * np.log(Y_pred2)))

loss1 =  0.35667494393873245
loss2 =  2.3025850929940455


In [ ]:
# Softmax + CrossEntropy (logSoftmax + NLLLoss)
loss = nn.CrossEntropyLoss()

In [ ]:
# target is of size nBatch
# each element in target has to have 0 <= value < nClasses (0-2)
# Input is class, not one-hot
Y = Variable(torch.LongTensor([0]), requires_grad=False)

In [ ]:
# input is of size nBatch x nClasses = 1 x 4
# Y_pred are logits (not softmax)
Y_pred1 = Variable(torch.Tensor([[2.0, 1.0, 0.1]]))
Y_pred2 = Variable(torch.Tensor([[0.5, 2.0, 0.3]]))

In [ ]:
l1 = loss(Y_pred1, Y)
l2 = loss(Y_pred2, Y)

print("PyTorch Loss1 = ", l1.data, "\nPyTorch Loss2=", l2.data)

print("Y_pred1=", torch.max(Y_pred1.data, 1)[1])
print("Y_pred2=", torch.max(Y_pred2.data, 1)[1])

PyTorch Loss1 =  tensor(0.4170) 
PyTorch Loss2= tensor(1.8406)
Y_pred1= tensor([0])
Y_pred2= tensor([1])


In [ ]:
# target is of size nBatch
# each element in target has to have 0 <= value < nClasses (0-2)
# Input is class, not one-hot
Y = Variable(torch.LongTensor([2, 0, 1]), requires_grad=False)

# input is of size nBatch x nClasses = 2 x 4
# Y_pred are logits (not softmax)
Y_pred1 = Variable(torch.Tensor([[0.1, 0.2, 0.9],
                                 [1.1, 0.1, 0.2],
                                 [0.2, 2.1, 0.1]]))


Y_pred2 = Variable(torch.Tensor([[0.8, 0.2, 0.3],
                                 [0.2, 0.3, 0.5],
                                 [0.2, 0.2, 0.5]]))


In [ ]:
l1 = loss(Y_pred1, Y)
l2 = loss(Y_pred2, Y)

print("Batch Loss1 = ", l1.data, "\nBatch Loss2=", l2.data)

Batch Loss1 =  tensor(0.4966) 
Batch Loss2= tensor(1.2389)


## 04.Softmax Classifier Using MNIST


In [ ]:
# https://github.com/pytorch/examples/blob/master/mnist/main.py
from __future__ import print_function
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torchvision import datasets, transforms
from torch.autograd import Variable

In [ ]:
# Training settings
batch_size = 64

# MNIST Dataset
train_dataset = datasets.MNIST(root='./data/',
                               train=True,
                               transform=transforms.ToTensor(),
                               download=True)

test_dataset = datasets.MNIST(root='./data/',
                              train=False,
                              transform=transforms.ToTensor())

# Data Loader (Input Pipeline)
train_loader = torch.utils.data.DataLoader(dataset=train_dataset,
                                           batch_size=batch_size,
                                           shuffle=True)

test_loader = torch.utils.data.DataLoader(dataset=test_dataset,
                                          batch_size=batch_size,
                                          shuffle=False)

In [ ]:
class Net(nn.Module):

    def __init__(self):
        super(Net, self).__init__()
        self.l1 = nn.Linear(784, 520)
        self.l2 = nn.Linear(520, 320)
        self.l3 = nn.Linear(320, 240)
        self.l4 = nn.Linear(240, 120)
        self.l5 = nn.Linear(120, 10)

    def forward(self, x):
        x = x.view(batch_size, 784)  # Flatten the data (n, 1, 28, 28)-> (n, 784) == x = x.view(64, -1,2)
        x = F.relu(self.l1(x))
        x = F.relu(self.l2(x))
        x = F.relu(self.l3(x))
        x = F.relu(self.l4(x))
        return self.l5(x)


model = Net()

In [ ]:
criterion = nn.CrossEntropyLoss()
optimizer = optim.SGD(model.parameters(), lr=0.01, momentum=0.5)

In [ ]:
def train(epoch):
    model.train()
    for batch_idx, (data, target) in enumerate(train_loader):
        data, target = Variable(data), Variable(target)
        optimizer.zero_grad()
        output = model(data)
        loss = criterion(output, target)
        loss.backward()
        optimizer.step()
        if batch_idx % 10 == 0:
            print('Train Epoch: {} [{}/{} ({:.0f}%)]\tLoss: {:.6f}'.format(
                epoch, batch_idx * len(data), len(train_loader.dataset),
                100. * batch_idx / len(train_loader), loss.item()))

In [ ]:
def test():
    model.eval() # gradient를 계산하지 않겠다 -> 계산량을 줄여서 빠르게
    test_loss = 0
    correct = 0
    for data, target in test_loader:
        data, target = Variable(data, volatile=True), Variable(target)
        output = model(data)
        # sum up batch loss
        test_loss += criterion(output, target).item()
        # get the index of the max
        pred = output.data.max(1, keepdim=True)[1] # output 2차원, 1:dim=1(열방향), [1]: max확률랑 레이블위치가 나오는데 레이블위치 반환
        correct += pred.eq(target.data.view_as(pred)).cpu().sum() # eq: 같은지(1) 다른지(0), view_as : pred랑 dimension 계산할 수 있도록 맞춰주겠다
                                                                  # cpu: print로 찍으려면 무조건 cpu로 가져와야해서 강제로 가져옴

    test_loss /= len(test_loader.dataset)
    print('\nTest set: Average loss: {:.4f}, Accuracy: {}/{} ({:.0f}%)\n'.format(
        test_loss, correct, len(test_loader.dataset),
        100. * correct / len(test_loader.dataset)))

In [ ]:
for epoch in range(1, 10):
    train(epoch)
    test()

Train Epoch: 1 [0/60000 (0%)]	Loss: 2.306516
Train Epoch: 1 [640/60000 (1%)]	Loss: 2.307921
Train Epoch: 1 [1280/60000 (2%)]	Loss: 2.297262
Train Epoch: 1 [1920/60000 (3%)]	Loss: 2.294854
Train Epoch: 1 [2560/60000 (4%)]	Loss: 2.295487
Train Epoch: 1 [3200/60000 (5%)]	Loss: 2.302616
Train Epoch: 1 [3840/60000 (6%)]	Loss: 2.298371
Train Epoch: 1 [4480/60000 (7%)]	Loss: 2.296983
Train Epoch: 1 [5120/60000 (9%)]	Loss: 2.308605
Train Epoch: 1 [5760/60000 (10%)]	Loss: 2.300529
Train Epoch: 1 [6400/60000 (11%)]	Loss: 2.298345
Train Epoch: 1 [7040/60000 (12%)]	Loss: 2.301335
Train Epoch: 1 [7680/60000 (13%)]	Loss: 2.291287
Train Epoch: 1 [8320/60000 (14%)]	Loss: 2.301517
Train Epoch: 1 [8960/60000 (15%)]	Loss: 2.299700
Train Epoch: 1 [9600/60000 (16%)]	Loss: 2.301294
Train Epoch: 1 [10240/60000 (17%)]	Loss: 2.308201
Train Epoch: 1 [10880/60000 (18%)]	Loss: 2.293571
Train Epoch: 1 [11520/60000 (19%)]	Loss: 2.294895
Train Epoch: 1 [12160/60000 (20%)]	Loss: 2.302766
Train Epoch: 1 [12800/60000 (

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:6: UserWarning: volatile was removed and now has no effect. Use `with torch.no_grad():` instead.
  



Test set: Average loss: 0.0225, Accuracy: 6280/10000 (63%)

Train Epoch: 2 [0/60000 (0%)]	Loss: 1.364009
Train Epoch: 2 [640/60000 (1%)]	Loss: 1.294365
Train Epoch: 2 [1280/60000 (2%)]	Loss: 1.357647
Train Epoch: 2 [1920/60000 (3%)]	Loss: 1.216259
Train Epoch: 2 [2560/60000 (4%)]	Loss: 1.218378
Train Epoch: 2 [3200/60000 (5%)]	Loss: 1.090624
Train Epoch: 2 [3840/60000 (6%)]	Loss: 1.186896
Train Epoch: 2 [4480/60000 (7%)]	Loss: 0.949177
Train Epoch: 2 [5120/60000 (9%)]	Loss: 0.925847
Train Epoch: 2 [5760/60000 (10%)]	Loss: 0.980053
Train Epoch: 2 [6400/60000 (11%)]	Loss: 0.874449
Train Epoch: 2 [7040/60000 (12%)]	Loss: 0.798589
Train Epoch: 2 [7680/60000 (13%)]	Loss: 0.899113
Train Epoch: 2 [8320/60000 (14%)]	Loss: 0.790320
Train Epoch: 2 [8960/60000 (15%)]	Loss: 0.976146
Train Epoch: 2 [9600/60000 (16%)]	Loss: 0.758660
Train Epoch: 2 [10240/60000 (17%)]	Loss: 0.558771
Train Epoch: 2 [10880/60000 (18%)]	Loss: 0.795880
Train Epoch: 2 [11520/60000 (19%)]	Loss: 0.721403
Train Epoch: 2 [12

## 03. Toy Inception MNIST

In [ ]:
# https://github.com/pytorch/examples/blob/master/mnist/main.py
from __future__ import print_function
import argparse
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torchvision import datasets, transforms
from torch.autograd import Variable

In [ ]:
# Training settings
batch_size = 64

# MNIST Dataset
train_dataset = datasets.MNIST(root='./data/',
                               train=True,
                               transform=transforms.ToTensor(),
                               download=True)

test_dataset = datasets.MNIST(root='./data/',
                              train=False,
                              transform=transforms.ToTensor())

# Data Loader (Input Pipeline)
train_loader = torch.utils.data.DataLoader(dataset=train_dataset,
                                           batch_size=batch_size,
                                           shuffle=True)

test_loader = torch.utils.data.DataLoader(dataset=test_dataset,
                                          batch_size=batch_size,
                                          shuffle=False)

In [ ]:
class InceptionA(nn.Module):

    def __init__(self, in_channels):
        super(InceptionA, self).__init__()
        self.branch1x1 = nn.Conv2d(in_channels, 16, kernel_size=1)

        self.branch5x5_1 = nn.Conv2d(in_channels, 16, kernel_size=1)
        self.branch5x5_2 = nn.Conv2d(16, 24, kernel_size=5, padding=2)

        self.branch3x3dbl_1 = nn.Conv2d(in_channels, 16, kernel_size=1)
        self.branch3x3dbl_2 = nn.Conv2d(16, 24, kernel_size=3, padding=1)
        self.branch3x3dbl_3 = nn.Conv2d(24, 24, kernel_size=3, padding=1)

        self.branch_pool = nn.Conv2d(in_channels, 24, kernel_size=1)

    def forward(self, x):
        branch1x1 = self.branch1x1(x)

        branch5x5 = self.branch5x5_1(x)
        branch5x5 = self.branch5x5_2(branch5x5)

        branch3x3dbl = self.branch3x3dbl_1(x)
        branch3x3dbl = self.branch3x3dbl_2(branch3x3dbl)
        branch3x3dbl = self.branch3x3dbl_3(branch3x3dbl)

        branch_pool = F.avg_pool2d(x, kernel_size=3, stride=1, padding=1)
        branch_pool = self.branch_pool(branch_pool)

        outputs = [branch1x1, branch5x5, branch3x3dbl, branch_pool]
        return torch.cat(outputs, 1)

In [ ]:
class Net(nn.Module):

    def __init__(self):
        super(Net, self).__init__()
        self.conv1 = nn.Conv2d(1, 10, kernel_size=5)
        self.conv2 = nn.Conv2d(88, 20, kernel_size=5)

        self.incept1 = InceptionA(in_channels=10)
        self.incept2 = InceptionA(in_channels=20)

        self.mp = nn.MaxPool2d(2)
        self.fc = nn.Linear(1408, 10)

    def forward(self, x):
        in_size = x.size(0)
        x = F.relu(self.mp(self.conv1(x)))
        x = self.incept1(x)
        x = F.relu(self.mp(self.conv2(x)))
        x = self.incept2(x)
        x = x.view(in_size, -1)  # flatten the tensor
        x = self.fc(x)
        return F.log_softmax(x)

In [ ]:
model = Net()
optimizer = optim.SGD(model.parameters(), lr=0.01, momentum=0.5)

In [ ]:
def train(epoch):
    model.train()
    for batch_idx, (data, target) in enumerate(train_loader):
        data, target = Variable(data), Variable(target)
        optimizer.zero_grad()
        output = model(data)
        loss = F.nll_loss(output, target)
        loss.backward()
        optimizer.step()
        if batch_idx % 10 == 0:
            print('Train Epoch: {} [{}/{} ({:.0f}%)]\tLoss: {:.6f}'.format(
                epoch, batch_idx * len(data), len(train_loader.dataset),
                100. * batch_idx / len(train_loader), loss.item()))

In [ ]:
def test():
    model.eval()
    test_loss = 0
    correct = 0
    for data, target in test_loader:
        data, target = Variable(data, volatile=True), Variable(target)
        output = model(data)
        # sum up batch loss
        test_loss += F.nll_loss(output, target, size_average=False).item()
        # get the index of the max log-probability
        pred = output.data.max(1, keepdim=True)[1]
        correct += pred.eq(target.data.view_as(pred)).cpu().sum()

    test_loss /= len(test_loader.dataset)
    print('\nTest set: Average loss: {:.4f}, Accuracy: {}/{} ({:.0f}%)\n'.format(
        test_loss, correct, len(test_loader.dataset),
        100. * correct / len(test_loader.dataset)))

In [ ]:
for epoch in range(1, 10):
    train(epoch)
    test()

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:22: UserWarning: Implicit dimension choice for log_softmax has been deprecated. Change the call to include dim=X as an argument.


Train Epoch: 1 [0/60000 (0%)]	Loss: 2.298459
Train Epoch: 1 [640/60000 (1%)]	Loss: 2.302282
Train Epoch: 1 [1280/60000 (2%)]	Loss: 2.314808
Train Epoch: 1 [1920/60000 (3%)]	Loss: 2.313843
Train Epoch: 1 [2560/60000 (4%)]	Loss: 2.307911
Train Epoch: 1 [3200/60000 (5%)]	Loss: 2.301989
Train Epoch: 1 [3840/60000 (6%)]	Loss: 2.292517
Train Epoch: 1 [4480/60000 (7%)]	Loss: 2.291605
Train Epoch: 1 [5120/60000 (9%)]	Loss: 2.301653
Train Epoch: 1 [5760/60000 (10%)]	Loss: 2.293882
Train Epoch: 1 [6400/60000 (11%)]	Loss: 2.293476
Train Epoch: 1 [7040/60000 (12%)]	Loss: 2.288318
Train Epoch: 1 [7680/60000 (13%)]	Loss: 2.307489
Train Epoch: 1 [8320/60000 (14%)]	Loss: 2.282997
Train Epoch: 1 [8960/60000 (15%)]	Loss: 2.300429
Train Epoch: 1 [9600/60000 (16%)]	Loss: 2.278991
Train Epoch: 1 [10240/60000 (17%)]	Loss: 2.284487
Train Epoch: 1 [10880/60000 (18%)]	Loss: 2.271263
Train Epoch: 1 [11520/60000 (19%)]	Loss: 2.272692
Train Epoch: 1 [12160/60000 (20%)]	Loss: 2.256499
Train Epoch: 1 [12800/60000 (

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:6: UserWarning: volatile was removed and now has no effect. Use `with torch.no_grad():` instead.
  
/usr/local/lib/python3.7/dist-packages/torch/nn/_reduction.py:42: UserWarning: size_average and reduce args will be deprecated, please use reduction='sum' instead.
  warnings.warn(warning.format(ret))



Test set: Average loss: 0.1995, Accuracy: 9402/10000 (94%)

Train Epoch: 2 [0/60000 (0%)]	Loss: 0.258538
Train Epoch: 2 [640/60000 (1%)]	Loss: 0.217049
Train Epoch: 2 [1280/60000 (2%)]	Loss: 0.169010
Train Epoch: 2 [1920/60000 (3%)]	Loss: 0.160412
Train Epoch: 2 [2560/60000 (4%)]	Loss: 0.413642
Train Epoch: 2 [3200/60000 (5%)]	Loss: 0.295826
Train Epoch: 2 [3840/60000 (6%)]	Loss: 0.261276
Train Epoch: 2 [4480/60000 (7%)]	Loss: 0.092610
Train Epoch: 2 [5120/60000 (9%)]	Loss: 0.242168
Train Epoch: 2 [5760/60000 (10%)]	Loss: 0.269855
Train Epoch: 2 [6400/60000 (11%)]	Loss: 0.191170
Train Epoch: 2 [7040/60000 (12%)]	Loss: 0.146693
Train Epoch: 2 [7680/60000 (13%)]	Loss: 0.315070
Train Epoch: 2 [8320/60000 (14%)]	Loss: 0.197020
Train Epoch: 2 [8960/60000 (15%)]	Loss: 0.235060
Train Epoch: 2 [9600/60000 (16%)]	Loss: 0.152280
Train Epoch: 2 [10240/60000 (17%)]	Loss: 0.170992
Train Epoch: 2 [10880/60000 (18%)]	Loss: 0.221109
Train Epoch: 2 [11520/60000 (19%)]	Loss: 0.164026
Train Epoch: 2 [12